We've checked out statistical approaches to language models in the last notebook. Now let's go find out what deep learning has to offer.

This time we build a language model that's character-level, not word level. 

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
!which python

/home/vscode/venv/bin/python


In [1]:
!pwd

/workspaces/nlp_cook_dishes/notebooks


Working on character level means that we don't need to deal with large vocabulary or missing words. Heck, we can even keep uppercase words in text! The downside, however, is that all our sequences just got a lot longer.

However, we still need special tokens:
* Begin Of Sequence  (__BOS__) - this token is at the start of each sequence. We use it so that we always have non-empty input to our neural network. $P(x_t) = P(x_1 | BOS)$
* End Of Sequence (__EOS__) - you guess it... this token is at the end of each sequence. The catch is that it should __not__ occur anywhere else except at the very end. If our model produces this token, the sequence is over.

In [2]:
import pandas as pd
import numpy as np
import ast


def clean_steps(data):
    data_list = ast.literal_eval(data)
    return ', '.join(data_list)


BOS, EOS = ' ', '\n'
data = pd.read_csv('RAW_recipes.csv')
data = data[['name', 'steps', 'description', 'ingredients']]
data = data[~data['steps'].str.contains('please ignore')]
data = data[~data['steps'].str.contains('none')]
data['steps'] = data['steps'].apply(clean_steps)
data['ingredients'] = data['ingredients'].apply(clean_steps)

lines = data.apply(lambda row: str(row['name']) + 
                   '; ' + 
                   str(row['steps']).replace("\n", ' ') + 
                   '; ' + 
                   str(row['description']) + 
                   '; ' + 
                   str(row['ingredients'])[:512], axis=1) \
            .apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()

Our next step is __building char-level vocabulary__. Put simply, you need to assemble a list of all unique tokens in the dataset.

In [3]:
# Get all unique characters from lines (including capital letters and symbols)
tokens = set(''.join(lines))
tokens = sorted(tokens)
n_tokens = len(tokens)
print ('n_tokens = ',n_tokens)

n_tokens =  142


We can now assign each character with its index in tokens list. This way we can encode a string into a torch-friendly integer vector.

In [4]:
# dictionary of character -> its identifier (index in tokens list)
token_to_id = {token: idx for idx, token in enumerate(tokens)}

Our final step is to assemble several strings in a integer matrix with shape `[batch_size, text_length]`. 

The only problem is that each sequence has a different length. We can work around that by padding short sequences with extra `"EOS"` tokens or cropping long sequences. Here's how it works:

In [5]:
def to_matrix(lines, max_len=None, pad=token_to_id[EOS], dtype=np.int64):
    """Casts a list of lines into torch-digestable matrix"""
    max_len = max_len or max(map(len, lines))
    lines_ix = np.full([len(lines), max_len], pad, dtype=dtype)
    for i in range(len(lines)):
        line_ix = list(map(token_to_id.get, lines[i][:max_len]))
        lines_ix[i, :len(line_ix)] = line_ix
    return lines_ix

In [6]:
#Example: cast 4 random names to a single matrix, pad with zeros where needed.
dummy_lines = [
    ' abc\n',
    ' abacaba\n',
    ' abc1234567890\n',
]
print(to_matrix(dummy_lines))

[[ 5 44 45 46  1  1  1  1  1  1  1  1  1  1  1]
 [ 5 44 45 44 46 44 45 44  1  1  1  1  1  1  1]
 [ 5 44 45 46 22 23 24 25 26 27 28 29 30 21  1]]


### Neural Language Model 

Just like for N-gram LMs, we want to estimate probability of text as a joint probability of tokens (symbols this time).

$$P(X) = \prod_t P(x_t \mid x_0, \dots, x_{t-1}).$$ 

Instead of counting all possible statistics, we want to train a neural network with parameters $\theta$ that estimates the conditional probabilities:

$$ P(x_t \mid x_0, \dots, x_{t-1}) \approx p(x_t \mid x_0, \dots, x_{t-1}, \theta) $$


But before we optimize, we need to define our neural network. Let's start with a fixed-window (aka convolutional) architecture:

<img src='https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/fixed_window_lm.jpg' width=400px>


We can think of neural language models as neural classifiers. They classify prefix of a text into |V| classes, where the classes are vocabulary tokens.

Intuitively, neural Language Models do two things:

- **process context → model-specific**  
The main idea here is to get a vector representation for the previous context. Using this representation, a model predicts a probability distribution for the next token. This part could be different depending on model architecture (e.g., RNN, CNN, whatever you want), but the main point is the same - to encode context.  

- **generate a probability distribution for the next token → model-agnostic**  
Once a context has been encoded, usually the probability distribution is generated in the same way


When designing a CNN language model, keep in mind the following things:

- prevent information flow from future tokens  
To predict a token, a left-to-right LM has to use only previous tokens - make sure your CNN does not see anything but them! For example, you can shift tokens to the right by using padding.  

- do not remove positional information  
Differently from text classification, positional information is very important for language models. Therefore, do not use pooling (or be very careful in how you do it).  
 
- if you stack many layers, do not forget about residual connections  
If you stack many layers, it may difficult to train a very deep network well. To avoid this, use residual connections  

1. Initialize the embedding layer to transform token indices into embeddings.
2. Add zero padding on the left to prevent the model from seeing future tokens.
3. Implement the convolutional layers, ensuring they only consider the current and previous words.
4. Include residual connections if using multiple layers.
5. Add a final layer to transform the output to the desired size.
6. Implement the forward pass, ensuring the input and output dimensions are compatible with embeddings and convolutions.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class FixedWindowLanguageModel(nn.Module):
    def __init__(self, n_tokens, emb_size=16, hid_size=64, filter_size=5, stride_size=1):
        super().__init__()

        # Embedding layer
        self.emb = nn.Embedding(n_tokens, emb_size)

        # Left padding to prevent future information leakage
        total_padding_needed = sum([(filter_size - 1) for _ in range(3)])
        self.left_padding = total_padding_needed


        # Convolutional layers with padding to maintain sequence length
        self.convs = nn.Sequential(
            nn.ZeroPad2d((self.left_padding, 0, 0, 0)),
            nn.Conv1d(emb_size, hid_size, kernel_size=filter_size, stride=stride_size, padding=0),
            nn.ReLU(),
            nn.Conv1d(hid_size, hid_size, kernel_size=filter_size, stride=stride_size, padding=0),
            nn.ReLU(),
            nn.Conv1d(hid_size, n_tokens, kernel_size=filter_size, stride=stride_size, padding=0)
        )

    def forward(self, input_ix):
        # Embedding input tokens
        emb = self.emb(input_ix).permute(0, 2, 1)  # [batch_size, emb_size, sequence_length]

        # Applying convolutions
        conv_out = self.convs(emb)

        # Adjusting shape for output
        output = conv_out.permute(0, 2, 1)  # [batch_size, sequence_length, n_tokens]
        return output


In [12]:
dummy_model = FixedWindowLanguageModel(n_tokens=len(tokens))

dummy_input_ix = torch.as_tensor(to_matrix(dummy_lines))
dummy_logits = dummy_model(dummy_input_ix)

print('Weights:', tuple(name for name, w in dummy_model.named_parameters()))

Weights: ('emb.weight', 'convs.1.weight', 'convs.1.bias', 'convs.3.weight', 'convs.3.bias', 'convs.5.weight', 'convs.5.bias')


In [13]:
dummy_logits.shape

torch.Size([3, 15, 142])

In [14]:
assert isinstance(dummy_logits, torch.Tensor)
assert dummy_logits.shape == (len(dummy_lines), max(map(len, dummy_lines)), n_tokens)
assert np.all(np.isfinite(dummy_logits.data.cpu().numpy())), "inf/nan encountered"
assert not np.allclose(dummy_logits.data.cpu().numpy().sum(-1), 1), "please predict linear outputs"

In [15]:
# test for lookahead
dummy_input_ix_2 = torch.as_tensor(to_matrix([line[:3] + 'e' * (len(line) - 3) for line in dummy_lines]))
dummy_logits_2 = dummy_model(dummy_input_ix_2)

assert torch.allclose(dummy_logits[:, :3], dummy_logits_2[:, :3]), "your model's predictions depend on FUTURE tokens. " \
    " Make sure you don't allow any layers to look ahead of current token." \
    " You can also get this error if your model is not deterministic (e.g. dropout). Disable it for this test."

The output from above. Let's break down what each part of this output represents:

1. **Embedding Layer**:
   - `'emb.weight'`: This is the weight matrix of the embedding layer. Each row corresponds to the embedding of a token in your vocabulary.

2. **Convolutional Layers**:
   - `'convs.X.1.weight'` and `'convs.X.1.bias'`: These are the weights and biases for the convolutional layers. `X` varies from 0 to 2, representing each convolutional layer in your `self.convs` ModuleList.
   - `'convs.X.3.weight'` and `'convs.X.3.bias'`: These represent the weights and biases for the batch normalization layers following each convolutional layer.

3. **Residual Layers**:
   - `'residual_layers.X.weight'` and `'residual_layers.X.bias'`: These are the weights and biases for the linear layers in your `self.residual_layers` ModuleList, used to match the dimensions for the residual connections.

4. **Final Linear Layer**:
   - `'final_layer.weight'` and `'final_layer.bias'`: These are the weights and biases of the final linear layer that outputs the logits for each token in the vocabulary.

This output shows that all the expected parameters for model's layers are correctly instantiated and registered within the model. Each layer's weights and biases are part of the model's parameters, which will be learned during training. 

We can now tune our network's parameters to minimize categorical crossentropy over training dataset $D$:

$$ L = {\frac1{|D|}} \sum_{X \in D} \sum_{x_i \in X} - \log p(x_t \mid x_1, \dots, x_{t-1}, \theta) $$

As usual with with neural nets, this optimization is performed via stochastic gradient descent with backprop.  One can also note that minimizing crossentropy is equivalent to minimizing model __perplexity__, KL-divergence or maximizng log-likelihood.

In [16]:
def compute_mask(input_ix, eos_ix=token_to_id[EOS]):
    """ compute a boolean mask that equals "1" until first EOS (including that EOS) """
    return F.pad(torch.cumsum(input_ix == eos_ix, dim=-1)[..., :-1] < 1, pad=(1, 0, 0, 0), value=True)

print('matrix:\n', dummy_input_ix.numpy())
print('mask:', compute_mask(dummy_input_ix).to(torch.int32).cpu().numpy())
print('lengths:', compute_mask(dummy_input_ix).sum(-1).cpu().numpy())

matrix:
 [[ 5 44 45 46  1  1  1  1  1  1  1  1  1  1  1]
 [ 5 44 45 44 46 44 45 44  1  1  1  1  1  1  1]
 [ 5 44 45 46 22 23 24 25 26 27 28 29 30 21  1]]
mask: [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
lengths: [ 5  9 15]


In [17]:
def compute_loss(model, input_ix):
    input_ix = torch.as_tensor(input_ix, dtype=torch.int64)

    logits = model(input_ix[:, :-1])
    reference_answers = input_ix[:, 1:]

    # Compute the mask
    mask = compute_mask(input_ix)

    # Compute cross-entropy loss
    loss = F.cross_entropy(logits.transpose(1, 2), reference_answers, reduction='none')

    # Apply the mask - zero out losses where mask is 0
    masked_loss = loss * mask[:, 1:].to(loss.dtype)

    # Compute mean loss, but only over non-zero elements in the mask
    loss = masked_loss.sum() / mask[:, 1:].sum()

    return loss

Note that the mask is shifted one position to the right compared to logits and reference_answers to align correctly (excluding the first token which has no previous token to predict from). The `F.cross_entropy` function expects logits of shape `[batch_size, num_classes, sequence_length]` and reference answers of shape `[batch_size, sequence_length]`, hence the `logits.transpose(1, 2)`.

The loss is calculated for each token, then masked to exclude unwanted tokens (padding and subsequent `EOS` tokens), and finally, the mean loss is computed over all remaining (non-masked) tokens. This ensures that the loss is properly normalized only over the relevant tokens.

In [18]:
loss_1 = compute_loss(dummy_model, to_matrix(dummy_lines, max_len=15))
loss_2 = compute_loss(dummy_model, to_matrix(dummy_lines, max_len=16))
assert (np.ndim(loss_1) == 0) and (0 < loss_1 < 100), "loss must be a positive scalar"
assert torch.allclose(loss_1, loss_2), 'do not include  AFTER first EOS into loss. '\
    'Hint: use compute_mask. Beware +/-1 errors. And be careful when averaging!'